In [28]:
import sympy as sp
from sympy import Symbol, Eq
from IPython.display import display
import numpy as np
from ipywidgets import interact, FloatSlider


class Waterbomb5PDiscrete:
    def __init__(self, a_num=1000/4, b_num=1615/4, c_num=645/4, e_num=286/4, gamma_num=0.683, t_num=15):
        # Prevent display during __init__
        object.__setattr__(self, '_initialized', False)

        # Define label map first
        object.__setattr__(self, 'label_map', {
            'W_ur_lower': r'W_{\downarrow}^{\urcorner}',
            'W_ul_lower': r'W_{\downarrow}^{\ulcorner}',
            'W_ll_lower': r'W_{\downarrow}^{\llcorner}',
            'W_lr_lower': r'W_{\downarrow}^{\lrcorner}',
            'H_ur_lower': r'H_{\downarrow}^{\urcorner}',
            'H_ul_lower': r'H_{\downarrow}^{\ulcorner}',
            'H_ll_lower': r'H_{\downarrow}^{\llcorner}',
            'H_lr_lower': r'H_{\downarrow}^{\lrcorner}',
            'H_mr_lower': r'H_{\downarrow}^{\rightarrow}',
            'H_ml_lower': r'H_{\downarrow}^{\leftarrow}',
            'U_ur_lower': r'U_{\downarrow}^{\urcorner}',
            'U_ul_lower': r'U_{\downarrow}^{\ulcorner}',
            'U_ll_lower': r'U_{\downarrow}^{\llcorner}',
            'U_lr_lower': r'U_{\downarrow}^{\lrcorner}',
            'delta_theta_ur': r'\Delta \theta^{\urcorner}',
            'theta': r'\theta',
            'x_folded': r'x_{\text{folded}}',
            'y_folded': r'y_{\text{folded}}',
            'z_folded': r'z_{\text{folded}}',
            'R_x': r'R_x',
            'O_center_lower': r'O_{\downarrow}^{+}',
            'C_mr_lower': r'C_{\downarrow}^{\dashv}',
            'C_ml_lower': r'C_{\downarrow}^{\vdash}',
            'V_mr_lower': r'V_{\downarrow}^{\dashv}',
            'V_ml_lower': r'V_{\downarrow}^{\vdash}',
        })
        # Symbolic variables
        self.theta = sp.Symbol('theta', real=True)
        self.a, self.b, self.c, self.e, self.gamma, self.t = sp.symbols(r'a b c e \gamma t', real=True)
        
        self.variables_num = {
            self.a: a_num,
            self.b: b_num,
            self.c: c_num,
            self.e: e_num,
            self.gamma: gamma_num,
            self.t: t_num
        }

        

        self.alpha = sp.pi / 2 - self.gamma
        self.cx = sp.cos(self.theta)
        self.sx = sp.sin(self.theta)

        self.R_x = sp.Matrix([
            [1, 0, 0],
            [0, self.cx, -self.sx],
            [0, self.sx, self.cx]
        ])

        self.x_folded = self.a + self.e
        self.y_folded = sp.sqrt(
            (self.b**2 * sp.cos(self.gamma)**2) -
            self.a**2 * (1 - sp.sin(self.gamma))**2
        ) / sp.cos(self.gamma)

        self.z_folded = (self.a * (1 - sp.sin(self.gamma))) / sp.cos(self.gamma)

        num = (self.z_folded - self.c * sp.cos(self.gamma))**2 - self.y_folded**2
        den = (self.z_folded - self.c * sp.cos(self.gamma))**2 + self.y_folded**2
        self.theta_expr = -sp.pi + sp.acos(num / den)

        self.O_center_lower = sp.Matrix([0, 0, 0])
        self.C_mr_lower = sp.Matrix([self.e, 0, 0])
        self.C_ml_lower = sp.Matrix([-self.e, 0, 0])
        self.V_mr_lower = sp.Matrix([self.e + self.c * sp.sin(self.gamma), 0, self.c * sp.cos(self.gamma)])
        self.V_ml_lower = sp.Matrix([-self.e - self.c * sp.sin(self.gamma), 0, self.c * sp.cos(self.gamma)])

        self.U_ur_lower = sp.Matrix([self.x_folded, self.y_folded, self.z_folded])
        self.U_ul_lower = sp.Matrix([-self.x_folded, self.y_folded, self.z_folded])
        self.U_ll_lower = sp.Matrix([-self.x_folded, -self.y_folded, self.z_folded])
        self.U_lr_lower = sp.Matrix([self.x_folded, -self.y_folded, self.z_folded])

        self.delta_theta_ur = self.V_mr_lower - self.R_x @ self.U_ll_lower

        self.W_ur_lower = self.R_x @ -self.C_mr_lower + self.delta_theta_ur
        self.W_ul_lower = self.W_ur_lower.multiply_elementwise(sp.Matrix([-1, 1, 1]))
        self.W_ll_lower = self.W_ur_lower.multiply_elementwise(sp.Matrix([-1, -1, 1]))
        self.W_lr_lower = self.W_ur_lower.multiply_elementwise(sp.Matrix([1, -1, 1]))

        self.H_ur_lower = self.delta_theta_ur
        self.H_ul_lower = self.delta_theta_ur.multiply_elementwise(sp.Matrix([-1, 1, 1]))
        self.H_ll_lower = self.delta_theta_ur.multiply_elementwise(sp.Matrix([-1, -1, 1]))
        self.H_lr_lower = self.delta_theta_ur.multiply_elementwise(sp.Matrix([1, -1, 1]))

        self.H_mr_lower = sp.Matrix([self.a + 2*self.e + self.c * sp.sin(self.gamma), 0, self.c * sp.cos(self.gamma)])
        self.H_ml_lower = sp.Matrix([-self.a - 2*self.e - self.c * sp.sin(self.gamma), 0, self.c * sp.cos(self.gamma)])

        # Numerical variables
        self.W_ul_lower_num = np.array(self.W_ul_lower.subs(self.theta, self.theta_expr).evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.W_ur_lower_num = np.array(self.W_ur_lower.subs(self.theta, self.theta_expr).evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.W_ll_lower_num = np.array(self.W_ll_lower.subs(self.theta, self.theta_expr).evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.W_lr_lower_num = np.array(self.W_lr_lower.subs(self.theta, self.theta_expr).evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.H_ul_lower_num = np.array(self.H_ul_lower.subs(self.theta, self.theta_expr).evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.H_ur_lower_num = np.array(self.H_ur_lower.subs(self.theta, self.theta_expr).evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.H_ll_lower_num = np.array(self.H_ll_lower.subs(self.theta, self.theta_expr).evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.H_lr_lower_num = np.array(self.H_lr_lower.subs(self.theta, self.theta_expr).evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.H_mr_lower_num = np.array(self.H_mr_lower.subs(self.theta, self.theta_expr).evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.H_ml_lower_num = np.array(self.H_ml_lower.subs(self.theta, self.theta_expr).evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.U_ur_lower_num = np.array(self.U_ur_lower.evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.U_ul_lower_num = np.array(self.U_ul_lower.evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.U_ll_lower_num = np.array(self.U_ll_lower.evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.U_lr_lower_num = np.array(self.U_lr_lower.evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.delta_theta_ur_num = np.array(self.delta_theta_ur.subs(self.theta, self.theta_expr).evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.O_center_lower_num = np.array(self.O_center_lower.evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.C_mr_lower_num = np.array(self.C_mr_lower.evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.C_ml_lower_num = np.array(self.C_ml_lower.evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.V_mr_lower_num = np.array(self.V_mr_lower.evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.V_ml_lower_num = np.array(self.V_ml_lower.evalf(subs=self.variables_num), dtype=float).flatten().tolist()
        self.theta_num = np.array(self.theta_expr.subs(self.variables_num)).astype(np.float64).flatten().tolist()[0]
        
        self.crease_nodes = [
            self.C_mr_lower,  # 0
            self.C_ml_lower,  # 1
            self.U_ur_lower,  # 2
            self.U_ul_lower,  # 3
            self.U_lr_lower,  # 4
            self.U_ll_lower,  # 5
            self.V_mr_lower,  # 6
            self.V_ml_lower,  # 7
            self.W_ul_lower,  # 8
            self.H_ul_lower,  # 9
            self.H_ml_lower,  # 10
            self.W_ll_lower,  # 11
            self.H_ll_lower,  # 12
            self.H_ur_lower,  # 13
            self.W_ur_lower,  # 14
            self.H_mr_lower,  # 15
            self.H_lr_lower,  # 16
            self.W_lr_lower,  # 17
        ]
        self.crease_nodes_num = np.array([
            self.C_mr_lower_num, # 0
            self.C_ml_lower_num, # 1
            self.U_ur_lower_num, # 2
            self.U_ul_lower_num, # 3
            self.U_lr_lower_num, # 4
            self.U_ll_lower_num, # 5
            self.V_mr_lower_num, # 6
            self.V_ml_lower_num, # 7
            self.W_ul_lower_num, # 8
            self.H_ul_lower_num, # 9
            self.H_ml_lower_num, # 10
            self.W_ll_lower_num, # 11
            self.H_ll_lower_num, # 12
            self.H_ur_lower_num, # 13
            self.W_ur_lower_num, # 14
            self.H_mr_lower_num, # 15
            self.H_lr_lower_num, # 16
            self.W_lr_lower_num, # 17
            ], dtype=np.float32)
        
        self.connectivity_crease_lines = [
            [15, 13],  # 0
            [6, 14],   # 1
            [6, 2],    # 2
            [0, 2],    # 3
            [1, 3],    # 4
            [7, 3],    # 5
            [7, 8],    # 6
            [10, 9],   # 7
            [10, 12],  # 8
            [7, 11],   # 9
            [7, 5],    # 10
            [1, 5],    # 11
            [0, 4],    # 12
            [6, 4],    # 13
            [6, 17],   # 14
            [15, 16],  # 15
            [9, 8],    # 16
            [8, 3],    # 17
            [3, 2],    # 18
            [2, 14],   # 19
            [14, 13],  # 20
            [10, 7],   # 21
            [7, 1],    # 22
            [1, 0],    # 23
            [0, 6],    # 24
            [6, 15],   # 25
            [12, 11],  # 26
            [11, 5],   # 27
            [5, 4],    # 28
            [4, 17],   # 29
            [17, 16]   # 30
            ]
        self.connectivity_facets = [
            [6, 15, 13, 14],  # 0
            [6, 14, 2],       # 1
            [6, 2, 0],        # 2
            [0, 2, 3, 1],     # 3
            [7, 1, 3],        # 4
            [7, 3, 8],        # 5
            [7, 8, 9, 10],    # 6
            [7, 10, 12, 11],  # 7
            [7, 11, 5],       # 8
            [7, 5, 1],        # 9
            [1, 5, 4, 0],     # 10
            [6, 0, 4],        # 11
            [6, 4, 17],       # 12
            [6, 17, 16, 15]   # 13
        ]
    
        # Allow __getattribute__ display after init
        object.__setattr__(self, '_initialized', True)

    def __getattribute__(self, name):
        # Always allow these
        if name in ('label_map', '_initialized', '__dict__', '__class__', 'DisplayWrapper'):
            return object.__getattribute__(self, name)

        value = object.__getattribute__(self, name)

        if object.__getattribute__(self, '_initialized'):
            label_map = object.__getattribute__(self, 'label_map')
            if name in label_map:
                label = label_map[name]

                class DisplayWrapper:
                    def __init__(self, label, value):
                        self.label = label
                        self.value = value

                    def _repr_html_(self):
                        display(Eq(Symbol(self.label), self.value, evaluate=False))
                        return ''  # suppress value display

                    def __repr__(self):
                        return ''  # suppress terminal output too

                    def __getattr__(self, attr):
                        return getattr(self.value, attr)

                    def __call__(self, *args, **kwargs):
                        return self.value(*args, **kwargs)

                return DisplayWrapper(label, value)

        return value


In [29]:
wb = Waterbomb5PDiscrete()  
wb.H_ll_lower

Eq(H_{\downarrow}^{\llcorner}, Matrix([
[                                                                                                                   -a - c*sin(\gamma) - 2*e],
[                -a*(1 - sin(\gamma))*sin(theta)/cos(\gamma) - sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*cos(theta)/cos(\gamma)],
[-a*(1 - sin(\gamma))*cos(theta)/cos(\gamma) + c*cos(\gamma) + sqrt(-a**2*(1 - sin(\gamma))**2 + b**2*cos(\gamma)**2)*sin(theta)/cos(\gamma)]]))

In [30]:
wb.C_ml_lower

Eq(C_{\downarrow}^{\vdash}, Matrix([
[-e],
[ 0],
[ 0]]))

In [31]:
from ipywidgets import FloatSlider, FloatText, VBox, HBox, Button, interactive_output, HTML, jslink, Checkbox, Text, Label
from IPython.display import display
import k3d
import numpy as np
import trimesh
import cadquery as cq
from cadquery import exporters

class WaterbombPlot(Waterbomb5PDiscrete):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.plot = None
        self.mesh = None

    def plot_k3d(self):
        vertices = np.array(self.crease_nodes_num, dtype=np.float32)
        faces = []
        for facet in self.connectivity_facets:
            if len(facet) == 3:
                faces.append(facet)
                faces.append([facet[2], facet[1], facet[0]])
            elif len(facet) == 4:
                tri1 = [facet[0], facet[1], facet[2]]
                tri2 = [facet[0], facet[2], facet[3]]
                faces.append(tri1)
                faces.append([tri1[2], tri1[1], tri1[0]])
                faces.append(tri2)
                faces.append([tri2[2], tri2[1], tri2[0]])
            else:
                raise ValueError("Only triangles and quadrilaterals are supported.")
        faces = np.array(faces, dtype=np.uint32)
        return k3d.mesh(vertices=vertices, indices=faces, color=0xff7700, opacity=1.0, flat_shading=True)

    def export_geometry(self, a, b, c, e, gamma, format):
        filename_base = f"wb_a_{a:.1f}_b_{b:.1f}_c_{c:.1f}_e_{e:.1f}_gamma_{gamma:.3f}"
        verts = np.array(self.crease_nodes_num, dtype=np.float32)
        faces = self.connectivity_facets

        if format in ['obj', 'stl']:
            tri_faces = []
            for facet in faces:
                if len(facet) == 3:
                    tri_faces.append(facet)
                elif len(facet) == 4:
                    tri_faces.append([facet[0], facet[1], facet[2]])
                    tri_faces.append([facet[0], facet[2], facet[3]])
            mesh = trimesh.Trimesh(vertices=verts, faces=tri_faces, process=False)
            mesh.export(f"{filename_base}.{format}")

        elif format == 'step':
            cq_faces = []
            for facet in faces:
                try:
                    pts = [cq.Vector(*verts[i]) for i in facet]
                    wire = cq.Workplane("XY").polyline(pts + [pts[0]]).close()
                    cq_faces.append(wire.close().toPending().faces().val())
                except Exception:
                    pass
            compound = cq.Compound.makeCompound(cq_faces)
            exporters.export(compound, f"{filename_base}.step")

    def create_slider_field(self, min_val, max_val, step, value, description, decimals):
        slider = FloatSlider(
            min=min_val,
            max=max_val,
            step=step,
            value=value,
            readout_format=f'.{decimals}f',
            continuous_update=True,
            layout={'width': '300px'}
        )
        text = Text(value=f"{value:.{decimals}f}", layout={'width': '80px'})
        label = Label(value=description, layout={'width': '70px'})

        def on_text_change(change):
            val = change['new'].replace(',', '.')
            try:
                fval = float(val)
                if min_val <= fval <= max_val:
                    slider.value = fval
            except ValueError:
                pass

        def on_slider_change(change):
            text.value = f"{change['new']:.{decimals}f}"

        text.observe(on_text_change, names='value')
        slider.observe(on_slider_change, names='value')

        return slider, HBox([label, text, slider])

    def interact_k3d(self):
        self.plot = k3d.plot(grid_visible=True, camera_auto_fit=False, background_color=0xFFFFFF)
        self.plot.bounds = [[-1000, -2000, -2000], [2000, 2000, 2000]]
        self.plot.camera = [0, -1500, 1000, 0, 0, 0, 0, 0, 1]
        self.mesh = None

        dummy = k3d.mesh(
            vertices=np.array([[-2000, -2000, -1000], [2000, 2000, 2000], [2000, -2000, -2000]], dtype=np.float32),
            indices=np.array([[0, 1, 2]], dtype=np.uint32),
            color=0xffffff, opacity=0.0
        )
        self.plot += dummy

        a_slider, a_row = self.create_slider_field(0, 450, 0.1, 250.0, 'a [mm]', 1)
        b_slider, b_row = self.create_slider_field(0, 500, 0.1, 403.75, 'b [mm]', 1)
        c_slider, c_row = self.create_slider_field(0, 300, 0.1, 161.25, 'c [mm]', 1)
        e_slider, e_row = self.create_slider_field(0, 150, 0.1, 71.5, 'e [mm]', 1)
        gamma_slider, gamma_row = self.create_slider_field(0.0, np.pi/2, 0.001, 0.683, 'γ [rad]', 3)

        defaults = dict(a=250.0, b=403.75, c=161.25, e=71.5, gamma=0.683)
        status_label = HTML(value="", layout={'height': '30px'})
        show_checkbox = Checkbox(value=True, description='Show Geometry')

        def update(a, b, c, e, gamma):
            try:
                updated = WaterbombPlot(a_num=a, b_num=b, c_num=c, e_num=e, gamma_num=gamma)
                new_mesh = updated.plot_k3d()

                if self.mesh is not None:
                    try:
                        self.plot -= self.mesh
                    except ValueError:
                        pass  # It may already have been removed

                self.mesh = new_mesh
                self.crease_nodes_num = updated.crease_nodes_num
                self.connectivity_facets = updated.connectivity_facets

                if show_checkbox.value and self.mesh not in self.plot.objects:
                    self.plot += self.mesh

                status_label.value = "<span style='color:green;'>✅ Geometry updated</span>"

            except Exception as err:
                status_label.value = (
                    f"<span style='color:red;'>⚠️ Invalid parameters: a={a}, b={b}, c={c}, e={e}, γ={gamma:.3f}<br>{err}</span>"
                )

        def on_checkbox_change(change):
            if self.mesh is not None:
                if change['new'] and self.mesh not in self.plot.objects:
                    self.plot += self.mesh
                elif not change['new'] and self.mesh in self.plot.objects:
                    self.plot -= self.mesh

        show_checkbox.observe(on_checkbox_change, names='value')

        def on_reset_clicked(b):
            a_slider.value = defaults['a']
            b_slider.value = defaults['b']
            c_slider.value = defaults['c']
            e_slider.value = defaults['e']
            gamma_slider.value = defaults['gamma']
            status_label.value = ""

        def export_handler(ext):
            try:
                self.export_geometry(a_slider.value, b_slider.value, c_slider.value, e_slider.value, gamma_slider.value, ext)
                status_label.value = f"<span style='color:green;'>💾 Exported to {ext.upper()}</span>"
            except Exception as err:
                status_label.value = f"<span style='color:red;'>❌ Export error: {err}</span>"

        reset_btn = Button(description='Reset', button_style='info')
        export_obj_btn = Button(description='Export OBJ', button_style='success')
        export_stl_btn = Button(description='Export STL', button_style='warning')
        export_step_btn = Button(description='Export STEP', button_style='danger')

        reset_btn.on_click(on_reset_clicked)
        export_obj_btn.on_click(lambda b: export_handler("obj"))
        export_stl_btn.on_click(lambda b: export_handler("stl"))
        export_step_btn.on_click(lambda b: export_handler("step"))

        out = interactive_output(update, {
            'a': a_slider, 'b': b_slider, 'c': c_slider, 'e': e_slider, 'gamma': gamma_slider
        })

        controls = VBox([
            a_row, b_row, c_row, e_row, gamma_row,
            show_checkbox,
            HBox([reset_btn, export_obj_btn, export_stl_btn, export_step_btn]),
            status_label
        ])

        display(VBox([self.plot, controls, out]))

In [32]:
wb = WaterbombPlot()
wb.interact_k3d()

In [33]:
import k3d
import numpy as np

# Get data
points = wb.crease_nodes_num
connectivity = wb.connectivity_crease_lines

# Initialize plot
plot = k3d.plot()

# --- Only black crease lines ---
line_segments = []
for start, end in connectivity:
    line_segments += [points[start], points[end], [np.nan, np.nan, np.nan]]
line_segments = np.array(line_segments, dtype=np.float32)

plot += k3d.line(
    line_segments,
    shader='simple',
    width=0.01,
    color=0x000000  # black only
)

# --- Add crease nodes (green dots) ---
plot += k3d.points(
    points.astype(np.float32),
    point_size=20,
    color=0x008000,  # green
    shader='3d'
)

# --- Node labels (green, label box) ---
for idx, point in enumerate(points):
    plot += k3d.text(
        text=str(idx),
        position=point.astype(np.float32),
        color=0x008000,
        size=0.7,
        label_box=True
    )

# --- Line labels (blue at midpoints, label box) ---
for idx, (start, end) in enumerate(connectivity):
    midpoint = (points[start] + points[end]) / 2
    plot += k3d.text(
        text=str(idx),
        position=midpoint.astype(np.float32),
        color=0x0000FF,  # blue
        size=0.7,
        label_box=True
    )

# Display the plot
plot.display()

Output()

In [34]:
import numpy as np
import k3d
from IPython.display import display

class VectorLine3D:
    def __init__(self, vectors, colors=None):
        self.vectors = vectors
        # Use default or repeat as needed
        default_colors = [0xff0000, 0x00ff00, 0x0000ff, 0xffff00, 0x00ffff, 0xff00ff]
        if colors is None:
            self.colors = (default_colors * ((len(vectors) // len(default_colors)) + 1))[:len(vectors)]
        else:
            self.colors = colors
        self.plot = None

    def compute_point_at_y(self, p, d, y_target):
        """Return point along vector p + t*d where y == y_target"""
        if d[1] != 0:
            t = (y_target - p[1]) / d[1]
            return (p + t * d).reshape(1, 3).astype(np.float32)
        else:
            return None  # Parallel to y, cannot intersect at y_target

    def create_plot(self, y_values):
        self.plot = k3d.plot(grid_visible=True, axes_helper=1)

        # Plot vector segments
        for i, (p, d) in enumerate(self.vectors):
            line_points = np.array([p, p + d], dtype=np.float32)
            self.plot += k3d.line(line_points, color=self.colors[i], width=0.01)

        # Loop through given y levels
        for y in y_values:
            level_points = []
            colors_at_y = []

            for i, (p, d) in enumerate(self.vectors):
                y0, y1 = p[1], (p + d)[1]
                if min(y0, y1) <= y <= max(y0, y1):
                    pt = self.compute_point_at_y(p, d, y)
                    if pt is not None:
                        level_points.append(pt[0])
                        colors_at_y.append(self.colors[i])

            # Plot individual colored points
            for pt, col in zip(level_points, colors_at_y):
                self.plot += k3d.points(pt.reshape(1, 3), point_size=0.2, color=col)

            # Draw black line only if 2+ valid points exist at this level
            if len(level_points) >= 2:
                self.plot += k3d.line(np.array(level_points, dtype=np.float32), color=0x000000, width=0.02)

        display(self.plot)

In [35]:

import numpy as np
import k3d
from IPython.display import display

class ReinforcementVectors3D_y:
    def __init__(self, point_pairs, colors=None):
        # point_pairs = list of (start, end)
        self.vectors = [(p0, p1 - p0) for p0, p1 in point_pairs]
        self.ends = [p0 + (p1 - p0) for p0, p1 in point_pairs]
        default_colors = [0xff0000, 0x00ff00, 0x0000ff, 0xffff00, 0x00ffff, 0xff00ff]
        n = len(point_pairs)
        self.colors = (colors or (default_colors * ((n // len(default_colors)) + 1)))[:n]
        self.plot = None

    def compute_point_at_y(self, p, d, y_target):
        """Return intersection point along finite segment (p, p + d) with plane y = y_target"""
        y0, y1 = p[1], (p + d)[1]
        if min(y0, y1) <= y_target <= max(y0, y1) and d[1] != 0:
            t = (y_target - p[1]) / d[1]
            return (p + t * d).reshape(1, 3).astype(np.float32)
        else:
            return None

    def create_plot(self, y_values):
        self.plot = k3d.plot(grid_visible=True, axes_helper=1)

        # Plot vector segments
        for i, (p, d) in enumerate(self.vectors):
            line_points = np.array([p, p + d], dtype=np.float32)
            self.plot += k3d.line(line_points, color=self.colors[i], width=0.01)

        # Plot level lines at valid intersections
        for y in y_values:
            level_points = []
            colors_at_y = []

            for i, (p, d) in enumerate(self.vectors):
                pt = self.compute_point_at_y(p, d, y)
                if pt is not None:
                    level_points.append(pt[0])
                    colors_at_y.append(self.colors[i])

            # Plot colored points and connecting line
            for pt, col in zip(level_points, colors_at_y):
                self.plot += k3d.points(pt.reshape(1, 3), point_size=0.2, color=col)

            if len(level_points) >= 2:
                self.plot += k3d.line(np.array(level_points, dtype=np.float32), color=0x000000, width=0.02)

        display(self.plot)
    

# --------------------------
# Define reinforcement vectors in both directions
vectors_reinf_y = [
    (wb.crease_nodes_num[15], wb.crease_nodes_num[13]),
    (wb.crease_nodes_num[6], wb.crease_nodes_num[14]),
    (wb.crease_nodes_num[6], wb.crease_nodes_num[2]),
    (wb.crease_nodes_num[0], wb.crease_nodes_num[2]),
    (wb.crease_nodes_num[1], wb.crease_nodes_num[3]),
    (wb.crease_nodes_num[7], wb.crease_nodes_num[3]),
    (wb.crease_nodes_num[7], wb.crease_nodes_num[8]),
    (wb.crease_nodes_num[10], wb.crease_nodes_num[9]),
    (wb.crease_nodes_num[10], wb.crease_nodes_num[12]),
    (wb.crease_nodes_num[7], wb.crease_nodes_num[11]),
    (wb.crease_nodes_num[7], wb.crease_nodes_num[5]),
    (wb.crease_nodes_num[1], wb.crease_nodes_num[5]),
    (wb.crease_nodes_num[0], wb.crease_nodes_num[4]),
    (wb.crease_nodes_num[6], wb.crease_nodes_num[4]),
    (wb.crease_nodes_num[6], wb.crease_nodes_num[17]),
    (wb.crease_nodes_num[15], wb.crease_nodes_num[16]),
]

vectors_reinf_x = [
    (wb.crease_nodes_num[9], wb.crease_nodes_num[8]),
    (wb.crease_nodes_num[8], wb.crease_nodes_num[3]),
    (wb.crease_nodes_num[3], wb.crease_nodes_num[2]),
    (wb.crease_nodes_num[2], wb.crease_nodes_num[14]),
    (wb.crease_nodes_num[14], wb.crease_nodes_num[13]),
    (wb.crease_nodes_num[10], wb.crease_nodes_num[7]),
    (wb.crease_nodes_num[7], wb.crease_nodes_num[1]),
    (wb.crease_nodes_num[1], wb.crease_nodes_num[0]),
    (wb.crease_nodes_num[0], wb.crease_nodes_num[6]),
    (wb.crease_nodes_num[6], wb.crease_nodes_num[15]),
    (wb.crease_nodes_num[12], wb.crease_nodes_num[11]),
    (wb.crease_nodes_num[11], wb.crease_nodes_num[5]),
    (wb.crease_nodes_num[5], wb.crease_nodes_num[4]),
    (wb.crease_nodes_num[4], wb.crease_nodes_num[17]),
    (wb.crease_nodes_num[17], wb.crease_nodes_num[16]),
    (wb.crease_nodes_num[6], wb.crease_nodes_num[14]),
    (wb.crease_nodes_num[6], wb.crease_nodes_num[2]),
    (wb.crease_nodes_num[0], wb.crease_nodes_num[2]),
    (wb.crease_nodes_num[1], wb.crease_nodes_num[3]),
    (wb.crease_nodes_num[7], wb.crease_nodes_num[3]),
    (wb.crease_nodes_num[7], wb.crease_nodes_num[8]),
    (wb.crease_nodes_num[7], wb.crease_nodes_num[11]),
    (wb.crease_nodes_num[7], wb.crease_nodes_num[5]),
    (wb.crease_nodes_num[1], wb.crease_nodes_num[5]),
    (wb.crease_nodes_num[0], wb.crease_nodes_num[4]),
    (wb.crease_nodes_num[6], wb.crease_nodes_num[4]),
    (wb.crease_nodes_num[6], wb.crease_nodes_num[17])
]

In [36]:
# Compute slicing planes
spacing_x = 12.7
spacing_y = 16
c_nom = 10

Lx_folded = abs(wb.crease_nodes_num[12][0] - wb.crease_nodes_num[16][0])
Lx_flat = 2 * (wb.variables_num[wb.a] + 2 * wb.variables_num[wb.e] + wb.variables_num[wb.c])
n_bar_x = int(Lx_flat // spacing_x)
x_vec = np.linspace(-Lx_folded/2 + c_nom, Lx_folded/2 - c_nom, n_bar_x)

Ly_folded = abs(wb.crease_nodes_num[13][1] - wb.crease_nodes_num[16][1])
Ly_flat = 2 * wb.variables_num[wb.b]
n_bar_y = int(Ly_flat // spacing_y)
y_vec = np.linspace(-Ly_folded/2 + c_nom, Ly_folded/2 - c_nom, n_bar_y)

# Instantiate and plot
visualizer = ReinforcementVectors3D_y(vectors_reinf_y)
y_vec = np.linspace(-Ly_folded/2+c_nom, Ly_folded/2-c_nom, n_bar_y)
visualizer.create_plot(y_values=y_vec)



Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [41]:
import numpy as np
import k3d
from IPython.display import display

class ReinforcementVectors3D_x:
    def __init__(self, point_pairs, colors=None):
        # point_pairs = list of (start, end)
        self.vectors = [(p0, p1 - p0) for p0, p1 in point_pairs]
        default_colors = [0xff0000, 0x00ff00, 0x0000ff, 0xffff00, 0x00ffff, 0xff00ff]
        n = len(point_pairs)
        self.colors = (colors or (default_colors * ((n // len(default_colors)) + 1)))[:n]
        self.plot = None

    def compute_point_at_x(self, p, d, x_target):
        """Return intersection point along finite segment (p, p + d) with plane x = x_target"""
        x0, x1 = p[0], (p + d)[0]
        if min(x0, x1) <= x_target <= max(x0, x1) and d[0] != 0:
            t = (x_target - p[0]) / d[0]
            return (p + t * d).reshape(1, 3).astype(np.float32)
        else:
            return None

    def create_plot(self, x_values):
        self.plot = k3d.plot(grid_visible=True, axes_helper=1)

        # Plot original vector segments
        for i, (p, d) in enumerate(self.vectors):
            segment = np.array([p, p + d], dtype=np.float32)
            self.plot += k3d.line(segment, color=self.colors[i], width=0.01)

        # Loop through all x values of interest
        for x in x_values:
            valid_points = []
            colors_at_x = []

            for i, (p, d) in enumerate(self.vectors):
                pt = self.compute_point_at_x(p, d, x)
                if pt is not None:
                    valid_points.append(pt[0])
                    colors_at_x.append(self.colors[i])

            # Sort points by their y-coordinate
            if valid_points:
                valid_points = np.array(valid_points)
                sorted_indices = np.argsort(valid_points[:, 1])
                sorted_points = valid_points[sorted_indices]

                # Plot intersection points
                for pt, col in zip(sorted_points, np.array(colors_at_x)[sorted_indices]):
                    self.plot += k3d.points(pt.reshape(1, 3), point_size=0.2, color=int(col))

                # Connect sorted points with a black line
                if len(sorted_points) >= 2:
                    self.plot += k3d.line(sorted_points.astype(np.float32), color=0x000000, width=0.02)

        display(self.plot)

In [42]:
# Instantiate and plot
visualizer = ReinforcementVectors3D_x(vectors_reinf_x)
x_vec = np.linspace(-Lx_folded/2+c_nom, Lx_folded/2-c_nom, n_bar_x)
visualizer.create_plot(x_values=x_vec)


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [43]:
n_bar_x

87